<b> Capstone Project Part 2 </b>

<b> Web Scraping with Beautiful Soup </b>

In [ ]:
import threading
from queue import Queue
import time
import requests
import bs4
from bs4 import BeautifulSoup
from pprint import pprint
import csv
from tqdm import tqdm
from urllib.request import urlopen, urlretrieve
from multiprocessing.dummy import Pool  # This is a thread-based Pool
from multiprocessing import cpu_count

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
import seaborn as sns

In [ ]:


# prod_page_link = soup.find('a', {'class':'prodItemThumb_link'})['href']
# prod_title = soup.find('span', {'class': 'prodItemInfo_name'})['title']
# prod_variety = soup.find('span', {'class': 'prodItemInfo_varietal'}).text
# prod_origin = soup.find('span', {'class': 'prodItemInfo_originText'}).text
# prod_ratings = soup.find('span', {'class': 'averageRating_average'}).text
# prod_rating_average = soup.find('span', {'class': 'averageRating_average'}).text
# prod_price = soup.find('span', {'class': 'productPrice_price-regWhole'}).text
# prod_type = soup.find('li', {'class': 'icon icon-glass-red prodAttr_icon prodAttr_icon-redWine'})['title']


# prod_img_link = soup.find('img', {'class': 'prodItemThumb_image'})['src']

# pprint(prod_type)

In [ ]:
# product name
def prod_title(soup):
    l = []
    for item in soup.find_all('span', {'class': 'prodItemInfo_name'}):
        try:
            prod_title = item.text
        except:
            prod_title = None
        l.append(prod_title)
    return l

# product page link for further information and comments
def prodpage_link(soup):
    l=[]
    for item in soup.find_all('a', {'class':'prodItemThumb_link'}):
        try:
            prod_page_link = item['href']
        except:
            prod_page_link = None
        l.append(prod_page_link)
    return l

# product variety
def prod_variety(soup):
    l = []
    for item in soup.find_all('span', {'class': 'prodItemInfo_varietal'}):
        try:
            prod_variety = item.text
        except:
            prod_variety = None
        l.append(prod_variety)
    return l

# product origin

def prod_origin(soup):
    l = []
    for item in soup.find_all('span', {'class': 'prodItemInfo_originText'}):
        try:
            prod_origin = item.text
        except:
            prod_origin = None
        l.append(prod_origin)
    return l

# prod site rating
def prod_site_rating_count(soup):
    l = []
    for item in soup.find_all('span', {'class': 'averageRating_count'}):
        try:
            prod_ratings = item.text.strip('Ratings')
        except:
            prod_ratings = None
        l.append(prod_ratings)
    return l

# prod site rating average
def prod_site_rateaverage(soup):
    l = []
    for item in soup.find_all('span', {'class': 'averageRating_average'}):
        try:
            prod_rating_average = item.text
        except:
            prod_rating_average = None
        l.append(prod_rating_average)
    return l

# prod price

def prod_price(soup):
    l = []
    for item in soup.find_all('span', {'class': 'productPrice_price-regWhole'}):
        try:
            prod_price = item.text
        except:
            prod_price = None
        l.append(prod_price)
    return l

        
# get image
def get_img(soup):
    l = []
    a = soup.find_all('img', {'class': 'prodItemThumb_image'})
    b = prod_title(soup)
    for img, title in zip(a, b):

        try:
            prod_img_link = img['src']
        except:
            prod_img_link = None

        img_url = 'http://www.wine.com{}'.format(prod_img_link)
        dir_name = 'images'
        file_name = dir_name + '/{}.jpg'.format(title.replace(" ", '_').replace('/', '_')) 
        urlretrieve(img_url, file_name)
        l.append(img_url)
    return l

def other_rate(soup):
    l = []
    m = []
    uls = soup.find_all('ul', {'class': 'wineRatings_list'})

    for ul in uls:
        title = []
        review = []

        ti = ul.find_all("span",{'class':"wineRatings_initials"})
        re = ul.find_all("span",{'class':"wineRatings_rating"})
        
        for a, b in zip(ti, re):
            try:
                t = a.text
                r = b.text
            except:
                t = None
                r = None
            title.append(t)
            review.append(r)
        l.append(title)
        m.append(review)
    return l, m
    

In [ ]:
# wine description from the product page
def prod_info(soup_prod): 
# winery description of the wine
    try:
        desc = soup_prod.find('div', {'class': 'viewMoreModule_text'}).getText(separator = ' ')
    except:
        desc=None
    return desc

# winery name
def winery_name(soup_prod):
    try:
        w = soup_prod.find('h2', {'class':'pipWinery_headline'}).text
    except:
        w = None
    return w

# function to pull the reviewing authority initials, ratings ans the review text.
def critical_acc(soup_prod):
    initl = []
    ratel = []
    reviewl = []
    all_rew = soup_prod.find_all('div', {'class': 'pipProfessionalReviews_list'})
    for item in all_rew:
        try:
            pr_init = item.find('span', class_='wineRatings_initials').text
            pr_rate = item.find('span', class_='wineRatings_rating').text
            pr_review = item.find('div', {'class': 'pipSecContent_copy'}).text
        except:
            pr_init =  None
            pr_rate = None
            pr_review = None
        initl.append(pr_init)
        ratel.append(pr_rate)
        reviewl.append(pr_review)
    return initl, ratel, reviewl

In [ ]:
# requesting all items on each search page 
# def search_all_req(soup):
#     page_items = soup.find_all('div', {"class": "prodItem_wrap"})
    
#     title = [prod_title(item) for item in page_items]
#     page_link = [prodpage_link(item) for item in page_items]
#     variety = [prod_variety(item) for item in page_items]
#     origin = [prod_origin(item) for item in page_items]
#     rating_count = [prod_site_rating_count(item) for item in page_items]
#     rating_average = [prod_site_rateaverage(item) for item in page_items]
#     img_link = [get_img(item) for item in page_items]
    
#     return (page_link, title, variety, origin, rating_count, rating_average, img_link)


In [ ]:
# # getting all details from single product description pages
# def single_page_req(links):
#     p_info = []
#     w_name= []
#     rev_inital= []
#     rev_rate=[] 
#     review = []
#     for link in links:
#         url = 'http://www.wine.com{}'.format(link)
# #     information from each product page        
#         res = requests.get(url)
#         content = res.text
#         soup_prod = BeautifulSoup(content, 'html.parser')
#         p_info.append(prod_info(soup_prod))
#         w_name = winery_name(soup_prod)
#         rev_inital.append(critical_acc(soup_prod)[0])
#         rev_rate.append(critical_acc(soup_prod)[1])
#         review.append(critical_acc(soup_prod)[2])
#     return p_info, w_name, rev_inital, rev_rate, review
        

In [ ]:
# prof_rate = ['WS', 'RP', 'W&S', 'JH', 'CG', 'WE', 'WW', 'BH', 'JS', 'TP', 'D', 'V', 'JD']
# 'rate_WS', 'rate_RP', 'rate_W&S', 'rate_JH', 'rate_CG', 
#                      'rate_WE', 'rate_WW', 'rate_BH', 'rate_JS', 'rate_TP', 'rate_D', 'rate_V', 'rate_JD',
#                     'rev_WS', 'rev_RP', 'rev_W&S', 'rev_JH', 'rev_CG', 'rev_WE', 'rev_WW', 'rev_BH', 'rev_JS', 
#                      'rev_TP', 'rev_D', 'rev_V', 'rev_JD'
# 'img_link',

In [ ]:
wlist1 = [('red-wine', 124, 3),('white-wine', 125, 3)]

In [ ]:
wlist = [('red-wine', 124, 430),('white-wine', 125, 174), ('rose-wine', 126, 18),('champagne-and-sparkling', 123, 30)]

def get_items(wtlist):
    csv_file = open('capstone_wine_data.csv', 'w')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['page_link','title', 'variety', 'origin', 'rating_count', 'rating_average', 'img_link',
                        'img_name', 'description', 'winery_name', 'other_rater', 'other_ratings'])
    
    URL_template = 'https://www.wine.com/list/wine/{}/7155-{}/{}?showOutOfStock=false'

    for item in tqdm(wtlist):
#         print('scraping {} type wines'.format(item[0]))

        for results in range(2, item[2]):
            
            try:
        #  connecting to the website for each type of wine
                URL = URL_template.format(item[0], item[1], results)
                response = requests.get(URL)
            # pulling the html string
                html = response.text
            # parsing html
                soup = BeautifulSoup(html, 'html.parser')
                print('Status Code: ',response.status_code)
                
                title = prod_title(soup)
                page_link = prodpage_link(soup)
                variety = prod_variety(soup)
                origin = prod_origin(soup) 
                rating_count = prod_site_rating_count(soup)
                rating_average = prod_site_rateaverage(soup)
                img_link = get_img(soup)
                other_raters, other_ratings = other_rate(soup)

                for i in range(len(page_link)):  
                    url = 'http://www.wine.com{}'.format(page_link[i])
                    res = requests.get(url)
                    t = res.text
                    soup_ = BeautifulSoup(t, 'html.parser')
                    description = prod_info(soup_)
                    wname = winery_name(soup_)
                    i_title = title[i].replace(" ", '_').replace('/', '_')
                    
                    csv_writer.writerow([title[i], page_link[i], variety[i], 
                                 origin[i], rating_count[i], rating_average[i], 
                                 img_link[i], i_title , description, 
                                 wname, other_raters[i], other_ratings[i]])
                    #if i%100 == 0:
                    
            except:
                #response.status_code !=200
                print('Failed')#response.status_code)
    csv_file.close()

In [ ]:
get_items(wlist)

In [ ]:
def get_items_(witem):
    csv_file = open('capstone_wine_data.csv', 'w')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['page_link','title', 'variety', 'origin', 'rating_count', 'rating_average', 'img_link',
                        'img_name', 'description', 'winery_name', 'other_rater', 'other_ratings'])
    
    URL_template = 'https://www.wine.com/list/wine/{}/7155-{}/{}?showOutOfStock=false'

#     for item in tqdm(wtlist):
    print('scraping {} type wines'.format(witem[0]))

    for results in range(2, witem[2]):
            
        try:
        #  connecting to the website for each type of wine
            URL = URL_template.format(witem[0], witem[1], results)
            response = requests.get(URL)
            # pulling the html string
            html = response.text
            # parsing html
            soup = BeautifulSoup(html, 'html.parser')
            print('Status Code: ',response.status_code)
                
            title = prod_title(soup)
            page_link = prodpage_link(soup)
            variety = prod_variety(soup)
            origin = prod_origin(soup) 
            rating_count = prod_site_rating_count(soup)
            rating_average = prod_site_rateaverage(soup)
            img_link = get_img(soup)
            other_raters, other_ratings = other_rate(soup)

            for i in range(len(page_link)):  
                url = 'http://www.wine.com{}'.format(page_link[i])
                res = requests.get(url)
                t = res.text
                soup_ = BeautifulSoup(t, 'html.parser')
                description = prod_info(soup_)
                wname = winery_name(soup_)
                i_title = title[i].replace(" ", '_').replace('/', '_')
                    
                csv_writer.writerow([title[i], page_link[i], variety[i], 
                                 origin[i], rating_count[i], rating_average[i], 
                                 img_link[i], i_title , description, 
                                 wname, other_raters[i], other_ratings[i]])
                    #if i%100 == 0:
                    
        except:
                #response.status_code !=200
            print('Failed')#response.status_code)
    csv_file.close()

In [ ]:
def write_items(results, writer):
    writer.writerow(results)

In [ ]:
def request_concurrent(wtlist):
    print("Requesting in parallel...\n")
    jobs = []
#     for item in tqdm(wtlist):
    thread = threading.Thread(name=wtlist, target=get_items, args=(wtlist,))
    jobs.append(thread)
    thread.start()
    print("Waiting for threads to finish execution.")
#     thread.join()
    for j in tqdm(jobs):
        j.join()


In [ ]:
request_concurrent(wlist)